# Credits

See main readme for credits.

In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os

from skimage.io import imread
from skimage.transform import resize

import data_utils

#import sys
#sys.path.append(os.path.join('.', '..')) # Allow us to import shared custom 
#                                         # libraries, like utils.py

# Tying everything together

Now that you have learned about the three most used network architectures: FFNs, CNNs and RNNs. It is time to combine these network types into a more advanced model. 

It often happens that you have a combination of data that cannot fully be modeled by one of these three types of network. Knowing how to divide the data into the right subsets, and then build a network that handles each subset efficiently can mean the difference between a great model and an unusable one. 

## Kaggle challenge

In this lab we will work on a data science challenge from [`kaggle.com`](kaggle.com).
Kaggle is a website to participate in real life challenges. Early 2017 it was bought by Google, who wanted access to the global community of data scientists it has created over the last 7 years. Since then Google have sponsored its expansion and now the prizes of the competitions and the amount of public datasets are bigger than ever. Most competitions on Kaggle have a dataset, an accuracy metric and a leaderboard to compare submissions.
You can read more about Kaggle [here](https://www.kaggle.com/about) and access a great amount of public datasets [here](https://www.kaggle.com/datasets).

**NB**: You will need a Kaggle account for this exercise!

The challenge we will pursue is the [_Leaf Classification_](https://www.kaggle.com/c/leaf-classification) challenge.
The dataset consists approximately 1,584 images of leaf specimens (16 samples each of 99 species) which have been converted to binary black leaves against white backgrounds. Three sets of features are also provided per image: a shape contiguous descriptor, an interior texture histogram, and a ﬁne-scale margin histogram. For each feature, a 64-attribute vector is given per leaf sample.

We will primarily look into the type of neural network best suited for handling this type of data. 
 * For images, usually the convolutional neural network does a pretty good job, 
 * For timeseries (like the shape) usually the RNN is the network of choice
 * For the describing features a FFN is often a great option

Lastly, we will train the model and put the outputs in a submission file that we can submit to kaggle.

# Get set up

Go to kaggle, create a user, download the dataset.
Unpack the dataset in the current directory.
You should now be good to go.

## Visualizing the data

First we start out by looking at the images. You need to download them first!

In [ ]:
image_paths = glob.glob("images/*.jpg")
print("Amount of images =", len(image_paths))

Then we load in the training data, which is in CSV format. For this, we use [pandas](https://pandas.pydata.org/). Pandas is a horrible tool, but useful for data analysis. Please refrain from using pandas in any production code.

In [ ]:
# now loading the train.csv to find features for each training point
train = pd.read_csv('train.csv')
train_images = ['images/{}.jpg'.format(i) for i in train.id.values]
# notice how we "only" have 990 (989+0 elem) images for training, the rest is for testing
train.tail()

With our training data and images loaded into memory. It is time to take a look at the data. Trying to classify leaves does not sound like a particularly difficult or interesting problem. We have probably all had teachers forcing us to do it on field trips as children.

But try to take a look at the 99 different categories and come up with a system that discern all 99 types of leaves from each other. 

In [ ]:
# First we find an example of each species in order to visualize it
species = np.array(sorted(train.species.unique()))
species_examples = [np.random.choice(train[train.species == s].id.values) for s in species]

# Then we gather its' index in our list of images in order to find the correct image
indexes = [image_paths.index('images/{}.jpg'.format(i)) for i in species_examples]

# now plot 1 image from each category
fig = plt.figure(figsize=(50, 50))
for i, idx in enumerate(indexes):
    plt.subplot(10, 10, i + 1)
    image = imread(image_paths[idx], as_grey=True)
    plt.imshow(image, cmap='gray')
    plt.title("%s" % (species[i]))
    plt.axis('off')
plt.show()

As you can see, classifying leaves is actually a very tough problem. What makes it even worse, is that we cannot use all the image data we have available. In order to decrease the amount of computations needed, we need to reduce the size of the images as much as possible. On top of that our neural network only accepts fixed size input tensors. This means we will have to resize the images to ensure they all have the same sizes.

This resizing is problematic because it alters the shape of the leaves, and for some of them, this is their most distinctive feature. Take a look at `Salix_Intergra` in the bottom left corner. Describing this leaf without taking its' shape into account seems extremely difficult.

In [ ]:
fig = plt.figure(figsize=(18, 6))
amount = 5
image_sample = np.random.choice(train_images, amount)

ax = plt.subplot(2, amount + 1, 1)
txt = ax.text(0.4, 0.5, 'Original', fontsize=20)
txt.set_clip_on(False)
plt.axis('off')
    
for i, path in enumerate(image_sample):
    plt.subplot(2, amount + 1, i + 2)
    image = imread(path, as_grey=True)
    plt.imshow(image, cmap='gray')
    _id = int(path.split('/')[-1].split('.')[0])
    plt.title("{0}\nshape: {1}".format(train[train.id == _id].species.values[0], image.shape))
    plt.axis('off')

ax = plt.subplot(2, amount + 1, len(image_sample) + 2)
txt = ax.text(0.4, 0.5, 'Resized', fontsize=20)
txt.set_clip_on(False)
plt.axis('off')
    
for i, path in enumerate(image_sample):
    i += len(image_sample) + 3
    plt.subplot(2, amount + 1, i)
    image = imread(path, as_grey=True)
    
    image = resize(image, output_shape=(300, 300)) # <-- This is the method that resizes the image
    
    plt.imshow(image, cmap='gray')
    
plt.show()

In [ ]:
# now do similar as in train example above for test.csv
test = pd.read_csv('test.csv')
# notice that we do not have species here, we need to predict that ..!
test.tail()

In [ ]:
# and now do similar as in train example above for test.csv
sample_submission = pd.read_csv('sample_submission.csv')
# accordingly to these IDs we need to provide the probability of a given plant being present
sample_submission.tail()

## Investigating the other features

Now that we have looked at the image data we have available, it is time to take a look at the other available features. Below we choose a random subset of the training data, and visualize the 3 types of available features:
* margin
* shape
* texture

Try to run it a few times to try and get an understanding of how the features differ from species to species.

In [ ]:
# try and extract and plot columns
X = train.as_matrix()
species = X[:, 1:2]
margin = X[:, 2:66]
shape = X[:, 66:130]
texture = X[:, 130:]

# let us plot some of the features
plt.figure(figsize=(21,7)) # Set the plot size
amount = 5 # Choose the amount of images we want to show at a time
for i, idx in enumerate(np.random.choice(range(len(train)), amount)):
    ax = plt.subplot(amount,4,1+i*4)
    txt = ax.text(0.2, 0.2, species[idx][0], fontsize=20)
    txt.set_clip_on(False)
    plt.axis('off')
    if i == 0:
        plt.title('Species', fontsize=20)
    plt.subplot(amount,4,2+i*4)
    plt.plot(margin[idx])
    if i == 0:
        plt.title('Margin', fontsize=20)
    plt.axis('off')
    plt.subplot(amount,4,3+i*4)
    plt.plot(shape[idx])
    if i == 0:
        plt.title('Shape', fontsize=20)
    plt.axis('off')
    plt.subplot(amount,4,4+i*4)
    plt.plot(texture[idx])
    if i == 0:
        plt.title('Texture', fontsize=20)

plt.tight_layout()
plt.show()

# Exercise 1

1. Test various resizings of the image until you have found the smallest resizing of the image where you can still see differentiate between the images. Use `IMAGE_SHAPE=(?, ?, 1)` to reflect your choices.

So far we have learned about the feed forward neural network, the convolutional neural network and the recurrent neural network.
Given margin and texture are histograms, shape is a contigious value over a "time" dimension 

* How could Margin, Shape and Texture be represented for classification?

* Describe what network you would build and how you would represent the data points (image, margin, shape and texture).

In [ ]:
# dims = [128, 64, 32]
# _samples = np.random.choice(train_images, 10)
# print(len(_samples))
# for img in _samples:
#     for dim in dims:
#         print('{0}x{0}'.format(dim))

#         plt.imshow(resize(imread(img, as_gray=True), output_shape=(dim, dim)), cmap='gray')
#         plt.show()

# Defining the data loader

In [ ]:
# loading data and setting up constants
TRAIN_PATH = "train.csv"
TEST_PATH = "test.csv"
IMAGE_PATHS = glob.glob("images/*.jpg")
NUM_CLASSES = 99
IMAGE_SHAPE = (64, 64, 1)
NUM_FEATURES = 64 # for all three features, margin, shape and texture
# train holds both X (input) and t (target/truth)
data = data_utils.load_data(train_path=TRAIN_PATH, 
                            test_path=TEST_PATH,
                            image_paths=IMAGE_PATHS,
                            image_shape=IMAGE_SHAPE[:2])
# to visualize the size of the dimensions of the data
# print
print("@@@Shape checking of data sets@@@")
# print
print("TRAIN")
print("\timages\t%s%f" % (data.train['images'].shape, data.train['images'].mean()))
print("\tmargins\t%s\t%f" % (data.train['margins'].shape, data.train['margins'].mean()))
print("\tshapes\t%s\t%f" % (data.train['shapes'].shape, data.train['shapes'].mean()))
print("\ttextures%s\t%f" % (data.train['textures'].shape, data.train['textures'].mean()))
print("\tts\t %s" % (data.train['ts'].shape))
print("\twhile training, batch_generator will onehot encode ts to (batch_size, num_classes)")
# print()
print("TEST")
print("\timages\t%s\t%f" % (data.test['images'].shape, data.test['images'].mean())) 
print("\tmargins\t%s\t%f" % (data.test['margins'].shape, data.test['margins'].mean()))
print("\tshapes\t%s\t%f" % (data.test['shapes'].shape, data.test['shapes'].mean()))
print("\ttextures%s\t%f" % (data.test['textures'].shape, data.test['textures'].mean()))
print("\tids\t%s" % (data.test['ids'].shape))

# Batch Generator

While training, we will not directly access the entire dataset, instead we have a `batch_generator` function to give us inputs aligned with their targets/ids in a size that our model can handle in memory (batch\_size).

Furthermore, the `batch_generator` also handles validation splitting.

In [ ]:
dummy_batch_gen = data_utils.batch_generator(data, batch_size=64, num_classes=99, num_iterations=5e3, seed=42)
train_batch = next(dummy_batch_gen.gen_train())
valid_batch, i = next(dummy_batch_gen.gen_valid())
test_batch, i = next(dummy_batch_gen.gen_test())

print("TRAIN")
print("\timages,", train_batch['images'].shape)
print("\tmargins,", train_batch['margins'].shape)
print("\tshapes,", train_batch['shapes'].shape)
print("\ttextures,", train_batch['textures'].shape)
print("\tts,", train_batch['ts'].shape)
print()
print("VALID")
print("\timages,", valid_batch['images'].shape)
print("\tmargins,", valid_batch['margins'].shape)
print("\tshapes,", valid_batch['shapes'].shape)
print("\ttextures,", valid_batch['textures'].shape)
print("\tts,", valid_batch['ts'].shape)
print()
print("TEST")
print("\timages,", test_batch['images'].shape)
print("\tmargins,", test_batch['margins'].shape)
print("\tshapes,", test_batch['shapes'].shape)
print("\ttextures,", test_batch['textures'].shape)
print("\tids,", len(test_batch['ids']))
# notice that mean is very different, which is why we use batch_norm in all input data in model

# Build the model

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, GRU, Conv2d, Dropout, MaxPool2d, BatchNorm1d
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax

In [ ]:
use_cuda = torch.cuda.is_available()

def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if use_cuda:
        return x.cuda()
    return x

def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()

In [ ]:
height, width, channels = IMAGE_SHAPE
conv_out_channels = 16
conv_stride = 1
pool_stride = 2

features_cat_size = 3 * NUM_FEATURES
cnn_final_size = 144
features_rnn_size = NUM_FEATURES+NUM_FEATURES+100

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # note 3x, because we concat the three feature inputs
        self.batchnorm_features = BatchNorm1d(num_features=features_cat_size)
        
#         self.dropout = Dropout(p=0.2)
#         # conv layer with (5, 5) kernel, stride of 1, and padding 2 (gives same dim output)
#         self.conv_1 = Conv2d(in_channels=channels,
#                              out_channels=conv_out_channels,
#                              kernel_size=5,
#                              stride=conv_stride,
#                              padding=2)
#         self.conv_2 = Conv2d(in_channels=conv_out_channels,
#                              out_channels=conv_out_channels,
#                              kernel_size=5,
#                              stride=conv_stride,
#                              padding=2)
#         self.conv_3 = Conv2d(in_channels=conv_out_channels,
#                              out_channels=conv_out_channels,
#                              kernel_size=5,
#                              stride=conv_stride,
#                              padding=2)
#         self.conv_4 = Conv2d(in_channels=conv_out_channels,
#                              out_channels=conv_out_channels,
#                              kernel_size=5,
#                              stride=conv_stride,
#                              padding=2)
#         self.pool_1 = MaxPool2d(kernel_size=3,
#                                 stride=pool_stride,
#                                 padding=0)
#         self.pool_2 = MaxPool2d(kernel_size=3,
#                                 stride=pool_stride,
#                                 padding=0)
#         self.pool_3 = MaxPool2d(kernel_size=3,
#                                 stride=pool_stride,
#                                 padding=0)
#         self.pool_4 = MaxPool2d(kernel_size=3,
#                                 stride=pool_stride,
#                                 padding=0)
#         self.batchnorm_image_cnn = BatchNorm1d(num_features=cnn_final_size)
        
#         self.rnn_1 = GRU(input_size=1,
#                          hidden_size=100,
#                          num_layers=1,
#                          batch_first=True)
#         self.batchnorm_shape = BatchNorm1d(num_features=features_rnn_size)
        
        self.l_out = Linear(in_features=features_cat_size,
                            out_features=NUM_CLASSES,
                            bias=False)
        
    def forward(self, x_img, x_margin, x_shape, x_texture):
        out = {}
        ## use concatenated leaf features only
        # concat the features
        x = torch.cat((x_margin, x_shape, x_texture), dim=1)
        features = self.batchnorm_features(x)
        
        ## image features only
        # we permute the dimensions, such that we have NCHW (num, channels, heigh, width)
#         x = x_img.permute(0, 3, 1, 2)
#         x = relu(self.pool_1(self.conv_1(x)))
#         x = relu(self.pool_2(self.conv_2(x)))
#         x = relu(self.pool_3(self.conv_3(x)))
#         x = self.dropout(x)
#         x = relu(self.pool_4(self.conv_4(x)))
#         features = x.view(BATCH_SIZE, -1)
#         features = self.batchnorm_image_cnn(features)
        
        ## use concatenated leaf features only, where shape has been through an rnn
        # use last hidden state of rnn
        #x_shape = self.batchnorm_shape(x_shape)
#         x_shape.unsqueeze_(-1)
#         _, hs = self.rnn_1(x_shape)
#         # note, hs gives (num_layers * num_directions, batch, hidden_size)
#         # so, we need to move batch dim to first element
#         hs = hs.permute(1, 0, 2)
#         hs = hs.view(BATCH_SIZE, -1)
#         x = torch.cat((x_margin, hs, x_texture), dim=1)
#         features = self.batchnorm_shape(x)
    
        out['out'] = softmax(self.l_out(features), dim=1)
        return out

net = Net()
if use_cuda:
    net.cuda()
print(net)

# Build the cost function

In [ ]:
BATCH_SIZE = 70
LEARNING_RATE = 0.001
criterion = nn.CrossEntropyLoss()
# weight_decay is equal to L2 regularization
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

def accuracy(ys, ts):
    predictions = torch.max(ys, 1)[1]
    correct_prediction = torch.eq(predictions, ts)
    return torch.mean(correct_prediction.float())

## Test network

In [ ]:
_img_shape = tuple([BATCH_SIZE] + list(IMAGE_SHAPE))
_feature_shape = (BATCH_SIZE, NUM_FEATURES)

def randnorm(size):
    return np.random.normal(0, 1, size).astype('float32')

# dummy data
_x_image = get_variable(Variable(torch.from_numpy(randnorm(_img_shape))))
_x_margin = get_variable(Variable(torch.from_numpy(randnorm(_feature_shape))))
_x_shape = get_variable(Variable(torch.from_numpy(randnorm(_feature_shape))))
_x_texture = get_variable(Variable(torch.from_numpy(randnorm(_feature_shape))))

# test the forward pass
output = net(x_img=_x_image, x_margin=_x_margin, x_shape=_x_shape, x_texture=_x_texture)
output['out']

# Train

In [ ]:
max_iter = 500
log_every = 10
VALIDATION_SIZE = 0.1 # 0.1 is ~ 100 samples for valition
eval_every = 20

def get_labels(batch):
    return get_variable(Variable(torch.from_numpy(batch['ts']).long()))

def get_input(batch):
    return {
        'x_img': get_variable(Variable(torch.from_numpy(batch['images']))),
        'x_margin': get_variable(Variable(torch.from_numpy(batch['margins']))),
        'x_shape': get_variable(Variable(torch.from_numpy(batch['shapes']))),
        'x_texture': get_variable(Variable(torch.from_numpy(batch['textures'])))
    }

train_loss, train_accs = [], []
batch_gen = data_utils.batch_generator(data,
                                       batch_size=BATCH_SIZE,
                                       num_classes=NUM_CLASSES,
                                       num_iterations=max_iter,
                                       seed=42,
                                       val_size=VALIDATION_SIZE)

net.train()
for i, batch_train in enumerate(batch_gen.gen_train()):
    if i % eval_every == 0:
        net.eval()
        val_losses, val_accs, val_lengths = 0, 0, 0
        for batch_valid, num in batch_gen.gen_valid():
            output = net(**get_input(batch_valid))
            labels_argmax = torch.max(get_labels(batch_valid), 1)[1]
            val_losses += criterion(output['out'], labels_argmax) * num
            val_accs += accuracy(output['out'], labels_argmax) * num
            val_lengths += num
        # divide by the total accumulated batch sizes
        val_losses /= val_lengths
        val_accs /= val_lengths
        print("### EVAL loss: {:.2f} accs: {:.2f}".format(get_numpy(val_losses)[0],
                                                          get_numpy(val_accs)[0]))
        net.train()
    
    output = net(**get_input(batch_train))
    labels_argmax = torch.max(get_labels(batch_train), 1)[1]
    batch_loss = criterion(output['out'], labels_argmax)
    
    train_loss.append(get_numpy(batch_loss))
    train_accs.append(get_numpy(accuracy(output['out'], labels_argmax)))
    
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()
    
    if i % log_every == 0:        
        print("train, it: {} loss: {:.2f} accs: {:.2f}".format(i, 
                                                               np.mean(train_loss), 
                                                               np.mean(train_accs)))
        # reset
        train_loss, train_accs = [], []
        
    if max_iter < i:
        break

# Submission to Kaggle

First we have to make testset predictions, then we have to place it in the submission file and the upload to kaggle for our score! You can upload at max 5 submissions a day.

In [ ]:
# GET PREDICTIONS
# containers to collect ids and predictions
ids_test, preds_test = [], []
net.eval()
# run like with validation
for batch_test, num in batch_gen.gen_test():
    output = net(**get_input(batch_test))
    y_out = output['out'].data

    ids_test += batch_test['ids']
    if num!=len(y_out):
        # in case of the last batch, num will be less than batch_size
        y_out = y_out[:num]
    preds_test.append(y_out)
preds_test = np.concatenate(preds_test, axis=0)
assert len(ids_test) == len(preds_test)

# Make submission file

In [ ]:
preds_df = pd.DataFrame(preds_test, columns=data.le.classes_)
ids_test_df = pd.DataFrame(ids_test, columns=["id"])
submission = pd.concat([ids_test_df, preds_df], axis=1)
submission.to_csv('submission_mlp.csv', index=False)

In [ ]:
# below prints the submission, can be removed and replaced with code block below
submission.head(5)

# Upload submission

1. Go to [`https://www.kaggle.com/c/leaf-classification/`](https://www.kaggle.com/c/leaf-classification/)
2. Make a submission
3. Click or drop your submission here (writing a description is good practice)
4. Submit

Success! 

# Exercise 2

1. Try to get the best score on Kaggle for this dataset as you can. (You can upload your solution multiple times as you progress)

A good implementation would get a score between $0.04$ to $0.06$ (the smaller the better), try and see if you can get there, and explain what might have gone wrong if you cant. 

When trying to improve the network nothing is sacred, you can change learning rate, try testing various learning rates, batch sizes, validation sizes, etc. And most importantly, the validation set is very small (only 1 sample per class), etc.

To get you of to a good start we have created a list of thing you might want to try:
* Include a second fully connected layer with dropout layer
* Try with L1 regularization
* Include L2 regularization (weigth decay)
* Use only the image for training (with CNN) - comment on the increased time between iterations.
* Use dropout between the convolutional layers
* Include the RNN part
* Increase or decrease the batch size 
* Change the image size to be rectangular, bigger or smaller
* Try to combine the FFN, CNN, RNN parts in various ways

If your network is not performing as well as you would like it to, [here](http://theorangeduck.com/page/neural-network-not-working) is a great explanation of what might have gone wrong.

## Insights?

What worked, and what didn't? Sometimes the fancy tricks might not do any good. Keep that in mind, and always start out simple.